## 실습 : 유튜브에 있는 영상의 제목 및 링크 수집하기

#### 주의 : 밑에 있는 폴더 위치, 타겟의 URL은 원하는 옵션에 맞게 수정을 할 것

In [41]:
# 라이브러리 불러오기

while(True) :
    try :
        import os  # 운영체제 관련 작업 (폴더 생성, 삭제 등)
        break
    except ModuleNotFoundError:
        !pip install os
    
while(True) :
    try :
        # 페이지 로딩 시간을 부여하는 라이브러리
        import time  # 시간 관련 기능 (대기, 지연)
        break
    except ModuleNotFoundError:
        !pip install time 
    
while(True) :
    try:
        # 컴퓨터용 브라우저
        # 컴퓨터용 키보드
        # 선택지 지정방식
        from selenium import webdriver as wb
        from selenium.webdriver.common.keys import Keys
        from selenium.webdriver.common.by import By 
        from selenium.webdriver.chrome.service import Service
        from selenium.webdriver.chrome.options import Options
        break
    except ModuleNotFoundError:
        !pip install selenium

while(True) :
    try :
        # 랭과 열로 이루어진 표와 같은 형태를 다룰 수 있는 라이브러리
        import pandas as pd 
        break
    except ModuleNotFoundError :
        !pip install pandas

print(f'필요한 라이브러리 import 완료')

필요한 라이브러리 import 완료


In [42]:
# 사용자의 환경마다 수정이 필요한 변수들

# 원하는 폴더의 위치
folder_locate = "C:\\Users\\smhrd1\\Documents\\GitHub\\Crawling\\Study"

# 수집을 원하는 유튜버의 동영상목록 링크 
target_video_lint = 'https://www.youtube.com/@ITSUB/videos'

In [43]:
# 전체적으로 관리할 변수를 관리


# 파일을 저장할 폴더의 위치  
ORIGIN_URL = folder_locate
# 파일을 저장할 폴더를 이름
FOLDER_NAME = 'youtube_list'


# 내가 수집하는 유튜브의 동영상 링크 
YOUTUBE_URL = target_video_lint

# 내가 수집하는 유투브의 이름(자동으로 처리)
YOUTUBER_NAME = YOUTUBE_URL.split('@')[-1].split('/')[0]

# 수집하려는 영상 제목의 ID값
VIDEO_TITLE_ID = 'video-title-link'
# 수집하려는 영상 링크의 ID값
VIDEO_URL_ID = 'video-title-link'

# csv로 저장하려는 파일의 이름
FILE_NAME = f'{YOUTUBER_NAME}' + '_youtube_list'

# 최종적으로 가지고 있을 유튜브의 정보들
# 영상의 이름들을 보관할 리스트
get_all_title = []
# 영상의 링크들을 보관할 리스트
get_all_url = []

# 웹 브라우저를 가지고 있을 변수
driver = None

In [44]:
# 크롬을 시작하고, 사이트에 접속할 함수
def web_start() :
    global driver
    
    chrome_options = Options()
    
    # 브라우저 배율을 25%로 설정
    chrome_options.add_argument("--force-device-scale-factor=0.25") 

    # 크롬 드라이버 실행
    driver = wb.Chrome(options=chrome_options)
    
    driver.get(YOUTUBE_URL)
    time.sleep(3)

In [45]:
# input값(ID)를 통해 모든 elements를 수집하는 함수
def find_elements_ID(input) : 
    try :
        fint_result = driver.find_elements(By.ID, input)
        return fint_result
    except :
        return 0 

# 페이지를 끝까지 스크롤하는 함수
def page_Scroll() : 
    while(True) :
        origin_title = find_elements_ID(VIDEO_TITLE_ID) # 페이지 검사를 위한 사전 타이틀 수 파악
        body = driver.find_element(By.TAG_NAME, 'body') #페이지의 바디를 찾기
        body.send_keys(Keys.END) # 바디에서 스크롤을 위한 'END'버튼 클릭
        time.sleep(0.5) # 페이지 로딩을 위한 0.5초 로딩 
        later_title = find_elements_ID(VIDEO_TITLE_ID)  #페이지 검사를 위한 타이틀 수 파악
        
        if(origin_title == later_title) : 
            print("페이지의 끝인지 검사중...")
            body.send_keys(Keys.END) # 바디에서 스크롤을 위한 'END'버튼 클릭
            time.sleep(2)
            check_title = find_elements_ID(VIDEO_TITLE_ID)  #페이지 검사를 위한 타이틀 수 파악
            if (check_title == later_title) :
                # 여전히 타이틀 수가 같으면
                print(f'페이지가 끝났습니다. 최종 영상의 수 : ', len(later_title))
                break
            else :
                print("페이지가 아직 끝나지 않아 스크롤을 다시 시작합니다. 현재 영상의 수 : ", len(check_title)) 

In [46]:
# 모든 타이틀을 수집하는 함수
def get_All_Title() : 
    temp_list = find_elements_ID(VIDEO_TITLE_ID) 
    temp_get_all_title = [i.text for i in temp_list] 
    for i in temp_get_all_title : 
        get_all_title.append(i) 
    print(f'{len(get_all_title)}개의 제목 데이터를 수집 완료')

In [47]:
# 모든 링크를 수집하는 함수
def get_All_Url() :
    temp_list = find_elements_ID(VIDEO_URL_ID) 
    temp_get_all_url = [element.get_attribute("href") for element in temp_list] 
    for i in temp_get_all_url :
        get_all_url.append(i)
    print(f'{len(get_all_url)}개의 제목 링크 데이터를 수집 완료')

In [48]:
# 폴더를 생성하고 이동하는 함수
def make_folder() :
    try :
        os.chdir(ORIGIN_URL)
        print(f'{ORIGIN_URL}로 이동 성공')
    except : 
        print(f'{ORIGIN_URL}로 이동 실패')
        return False
        
    try : 
        os.mkdir(FOLDER_NAME)
        print(f'{FOLDER_NAME} 폴더를 생성했씁니다.')
    except :
        print(f'{FOLDER_NAME} 폴더가 이미 있습니다.')

    # 생성했거나, 이미 있는 폴더로 이동
    os.chdir(FOLDER_NAME)
    
    return True

# csv 파일로 저장하는 함수
def make_csv() :
    make_folder_check = make_folder()
    if not make_folder_check :
        print(f'폴더 이동에 실패하여 지금 위치에 파일을 생성합니다. ')
    pd.set_option('display.max_colwidth', None)
    df = pd.DataFrame({'제목' : get_all_title, '링크' : get_all_url})
    df.index = df.index + 1  
    
    file_index = 0
    new_file_name = None
    while(True) :
        new_file_name = f"{FILE_NAME}_{file_index:04}.csv" 
        
        name_check = os.path.isfile(new_file_name)
        if (name_check) :
            # 동일명의 파일이 있다면
            file_index += 1
        else :
            # 파일이 없다면 
            break
    try :
        df.to_csv(new_file_name, encoding='utf-8') 
        print(f'{new_file_name} 파일을 성공적으로 생성했습니다.')
    except :
        print(f'{new_file_name} 파일 생성 실패...')

In [49]:
# 브라우저를 닫는 함수
def web_close() :
    driver.close() 
    print(f'시스템이 종료되었습니다.')

In [ ]:
web_start()
page_Scroll()
get_All_Title()
get_All_Url()
web_close()
make_csv() 

페이지의 끝인지 검사중...
페이지가 아직 끝나지 않아 스크롤을 다시 시작합니다. 현재 영상의 수 :  960
페이지의 끝인지 검사중...
페이지가 아직 끝나지 않아 스크롤을 다시 시작합니다. 현재 영상의 수 :  1168
페이지의 끝인지 검사중...
페이지가 끝났습니다. 최종 영상의 수 :  1337
1337개의 제목 데이터를 수집 완료
